In [ ]:
import os
import uuid

def search_and_join(directory, search_terms):
    matching_files = []
    FILENAME=""
    # Step 1: Search for files with the specified extension
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            matching_files.append(os.path.join(directory, filename))

    # Step 2: Join files containing all search terms
    unique = str(uuid.uuid4())
    output_filename = '_'.join(search_terms)+unique+'.txt'
    with open(output_filename, 'w') as output_file:
        for file_path in matching_files:
            with open(file_path, 'r') as input_file:
                file_content = input_file.read()

                # Check if all search terms are present in the file
                if all(term in file_content for term in search_terms):
                    output_file.write(f"\n=== Contents of {file_path} ===\n")
                    output_file.write(file_content)

    print(f"Joined files containing all search terms into {output_filename}")
    FILENAME=FILENAME+output_filename
    print('FILENAME:',FILENAME)

# Example usage:
search_terms = ['temp', '.mp4','shutil','output','3']
directory_to_search = '/home/jack/Desktop/flask_conversations2/CHATGPT/output_txt_html_json/'

search_and_join(directory_to_search, search_terms)


In [ ]:
!cat temp_.mp4_shutil_output_335a5b71f-0f82-481f-869d-9fa67043120c.txt

In [ ]:
!rm random_video_mp4.txt

In [ ]:
import os

def search_and_join(directory, search_terms, output_filename):
    matching_files = []

    # Step 1: Search for files with the specified extension
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            matching_files.append(os.path.join(directory, filename))

    # Step 2: Join files containing any of the search terms
    with open(output_filename, 'w') as output_file:
        for file_path in matching_files:
            with open(file_path, 'r') as input_file:
                file_content = input_file.read()

                # Check if any search term is present in the file
                if any(term in file_content for term in search_terms):
                    output_file.write(f"\n=== Contents of {file_path} ===\n")
                    output_file.write(file_content)

    print(f"Joined files containing the terms into {output_filename}")

# Example usage:
search_terms = ['concatenate']
directory_to_search = '/home/jack/Desktop/flask_conversations2/CHATGPT/output_txt_html_json/'
output_filename = 'random_video_mp4.txt'

search_and_join(directory_to_search, search_terms, output_filename)


In [ ]:
!cat random_video_mp4.txt

In [ ]:
!rm random_video_mp4.txt

In [ ]:
import os
import csv
import sqlite3
from moviepy.editor import VideoFileClip

def is_symbolic_link(path):
    return os.path.islink(path)

def process_video(video_path):
    try:
        video_info = VideoFileClip(video_path)
        return {
            'path_name': video_path,
            'file_size': os.path.getsize(video_path),
            'video_resolution': video_info.size,
            'fps': video_info.fps,
            'duration': video_info.duration,
            'status': 'good',
            'symlink': 'yes' if is_symbolic_link(video_path) else 'no'
        }
    except Exception as e:
        return {
            'path_name': video_path,
            'status': 'bad',
            'symlink': 'yes' if is_symbolic_link(video_path) else 'no'
        }

def create_database(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create a table to store video information
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS videos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            path_name TEXT,
            file_size INTEGER,
            video_resolution TEXT,
            fps REAL,
            duration REAL,
            status TEXT,
            symlink TEXT
        )
    ''')

    conn.commit()
    conn.close()

def insert_video_data(database_path, video_data):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Insert video data into the database
    cursor.execute('''
        INSERT INTO videos (path_name, file_size, video_resolution, fps, duration, status, symlink)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (video_data['path_name'], video_data['file_size'], video_data['video_resolution'],
          video_data['fps'], video_data['duration'], video_data['status'], video_data['symlink']))

    conn.commit()
    conn.close()

def process_video_list(input_file, database_path):
    create_database(database_path)

    with open(input_file, 'r') as file:
        video_paths = [line.strip() for line in file]

    for i, video_path in enumerate(video_paths, 1):
        video_data = process_video(video_path)
        insert_video_data(database_path, video_data)

        # Print a dot after every 250 files processed
        if i % 250 == 0:
            print('.', end='', flush=True)

if __name__ == "__main__":
    input_file_path = '/home/jack/Desktop/all.txt'
    database_path = '/home/jack/Desktop/video_database.db'

    process_video_list(input_file_path, database_path)


In [ ]:
!rm /home/jack/Desktop/video_database.db

In [19]:
import os
import csv
import sqlite3
import logging
from moviepy.editor import VideoFileClip


def configure_logging(log_file_path):
    # Configure logging to write to both file and console
    logging.basicConfig(
        filename=log_file_path,
        level=logging.ERROR,
        format='%(asctime)s [%(levelname)s]: %(message)s',
    )
    
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.ERROR)
    logging.getLogger().addHandler(console_handler)


def process_video(video_path):
    logging.info(f"Processing video: {video_path}")
    video_data = {'path_name': video_path}

    try:
        clip = VideoFileClip(video_path)
        video_data['fps'] = clip.fps
        video_data['duration'] = clip.duration
        video_data['video_resolution'] = clip.size
        video_data['status'] = 'Success'

        # Check if it's an image video
        if clip.duration is None:
            video_data['duration'] = 0
            video_data['status'] = 'Image Video'
    except Exception as e:
        logging.error(f"Error processing video {video_path}: {str(e)}")
        video_data['status'] = f'Error: {str(e)}'

    logging.info(f"Processed video data: {video_data}")
    return video_data
def insert_video_data(database_path, video_data):
    logging.info(f"Inserting video data into the database: {video_data}")
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    try:
        # Convert file_size to integer explicitly before binding
        video_data['file_size'] = os.path.getsize(video_data['path_name'])
        video_data['symlink'] = os.path.islink(video_data['path_name'])

        # Convert video_resolution to string before insertion
        video_data['video_resolution'] = str(video_data['video_resolution'])

        # Insert video data into the database
        cursor.execute('''
            INSERT INTO videos (path_name, file_size, video_resolution, fps, duration, status, symlink)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            video_data['path_name'],
            video_data['file_size'],
            video_data['video_resolution'],
            video_data['fps'],
            video_data['duration'],
            video_data['status'],
            video_data['symlink']
        ))

        conn.commit()
        logging.info("Video data inserted successfully.")
    except Exception as e:
        logging.error(f"Error inserting video data into the database: {str(e)}")
    finally:
        conn.close()


def process_video_list(input_file, database_path):
    logging.info(f"Processing video list from file: {input_file}")
    with open(input_file, 'r') as file:
        video_paths = file.read().splitlines()

    for i, video_path in enumerate(video_paths, 1):
        video_data = process_video(video_path)
        insert_video_data(database_path, video_data)

        # Print a dot after every 250 files processed
        if i % 250 == 0:
            logging.info(f"Processed {i} videos.")

    logging.info("Video processing completed.")


def is_symbolic_link(path):
    return os.path.islink(path)


def create_database(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create a table to store video information
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS videos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            path_name TEXT,
            file_size INTEGER,
            video_resolution TEXT,
            fps REAL,
            duration REAL,
            status TEXT,
            symlink TEXT
        )
    ''')

    conn.commit()
    conn.close()


if __name__ == "__main__":
    input_file_path = '/home/jack/Desktop/all.txt'
    database_path = '/home/jack/Desktop/video_database.db'
    error_log_file_path = '/home/jack/Desktop/error_log.txt'

    # Configure logging
    configure_logging(error_log_file_path)

    # Create the database table
    create_database(database_path)

    # Process the video list
    process_video_list(input_file_path, database_path)


Error processing video /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00029tent.mp4: MoviePy error: failed to read the duration of file /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00029tent.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --en

Error processing video /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00029tent.mp4: MoviePy error: failed to read the duration of file /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00029tent.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --en

Error processing video /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00008tent.mp4: MoviePy error: failed to read the duration of file /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00008tent.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --en

Error processing video /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00008tent.mp4: MoviePy error: failed to read the duration of file /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00008tent.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --en

Error processing video /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00014tent.mp4: MoviePy error: failed to read the duration of file /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00014tent.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --en

...

Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/576x768_combined_flip_books.mp4: MoviePy error: failed to read the duration of file /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/576x768_combined_flip_books.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --

Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/fish.mp4: 'video_fps'
Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/fish.mp4: 'video_fps'
Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/fish.mp4: 'video_fps'
Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/fish.mp4: 'video_fps'
Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/fish.mp4: 'video_fps'
Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/fish.mp4: 'video_fps'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting 

Error processing video /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/vids/women01aazz.mp4: MoviePy error: failed to read the duration of file /home/jack/Desktop/HDD500/FlaskToys_flask_transitions/vids/women01aazz.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --en

Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'
Error inserting video data into the database: 'video_resolution'


KeyboardInterrupt: 

In [21]:
def print_database_rows(database_path, start_rowid, end_rowid):
    logging.info(f"Printing database rows from rowid {start_rowid} to {end_rowid}")

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    try:
        # Select rows from the specified range
        cursor.execute('''
            SELECT ROWID, * FROM videos
            WHERE id BETWEEN ? AND ?
        ''', (start_rowid, end_rowid))

        rows = cursor.fetchall()

        if not rows:
            logging.info("No rows found in the specified range.")
        else:
            for row in rows:
                logging.info(f"Row {row[0]}: {row[1:]}")
    except Exception as e:
        logging.error(f"Error querying the database: {str(e)}")
    finally:
        conn.close()

# Call the function to print rows from rowid 30 to 39
database_path = '/home/jack/Desktop/video_database.db'
print_database_rows(database_path, 30, 39)


In [24]:
database_path = '/home/jack/Desktop/video_database.db'
conn = sqlite3.connect(database_path)
cursor = conn.cursor()
cnt =0
rows = cursor.execute('SELECT ROWID, * FROM videos')
for row in rows:
    cnt=cnt+1
    if cnt <20:
        print(cnt,row)
    
print(cnt)            

1 (1, 1, '/home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/video.mp4', 2928856, '[512, 768]', 24.0, 45.0, 'Success', '0')
2 (2, 2, '/home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/square_vids20230325-151851video.mp4', 2692293, '[640, 640]', 24.0, 30.0, 'Success', '0')
3 (3, 3, '/home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/square_vids20230325-151821video.mp4', 847479, '[640, 640]', 24.0, 11.0, 'Success', '0')
4 (4, 4, '/home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/temp.mp4', 5466534, '[640, 640]', 24.0, 50.0, 'Success', '0')
5 (5, 5, '/home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/webp_images/yodo.mp4', 1133136, '[512, 768]', 1.0, 27.0, 'Success', '0')
6 (6, 6, '/home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/stacked_vids/stacked_videos.mp4', 9914734, '[640, 1280]', 24.0, 50.0, 'Success', '0')
7 (7, 7, '/home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/stacked_vids/stacked_vids202

In [ ]:
def insert_video_data(database_path, video_data):
    logging.info(f"Inserting video data into the database: {video_data}")
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    try:
        # Convert file_size to integer explicitly before binding
        video_data['file_size'] = os.path.getsize(video_data['path_name'])
        video_data['symlink'] = os.path.islink(video_data['path_name'])

        # Convert video_resolution to string before insertion
        video_data['video_resolution'] = str(video_data['video_resolution'])

        # Insert video data into the database
        cursor.execute('''
            INSERT INTO videos (path_name, file_size, video_resolution, fps, duration, status, symlink)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            video_data['path_name'],
            video_data['file_size'],
            video_data['video_resolution'],
            video_data['fps'],
            video_data['duration'],
            video_data['status'],
            video_data['symlink']
        ))

        conn.commit()
        logging.info("Video data inserted successfully.")
    except Exception as e:
        logging.error(f"Error inserting video data into the database: {str(e)}")
    finally:
        conn.close()

In [15]:
!ls /home/jack/Desktop/error_log.txt

ls: cannot access '/home/jack/Desktop/error_log.txt': No such file or directory


In [18]:
!rm /home/jack/Desktop/error_log.txt
!rm /home/jack/Desktop/video_database.db

rm: cannot remove '/home/jack/Desktop/error_log.txt': No such file or directory


In [8]:
!cp /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00023tent.mp4 /home/jack/Desktop/HDD500/DOCKERfile_flask/YouTube_Video/static/exotic/00029tent.mp4

In [ ]:
def insert_video_data(database_path, video_data):
    logging.info(f"Inserting video data into the database: {video_data}")
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    try:
        # Convert file_size to integer explicitly before binding
        video_data['file_size'] = os.path.getsize(video_data['path_name'])
        video_data['symlink'] = os.path.islink(video_data['path_name'])

        # Convert video_resolution to string before insertion
        video_data['video_resolution'] = str(video_data['video_resolution'])

        # Insert video data into the database
        cursor.execute('''
            INSERT INTO videos (path_name, file_size, video_resolution, fps, duration, status, symlink)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            video_data['path_name'],
            video_data['file_size'],
            video_data['video_resolution'],
            video_data['fps'],
            video_data['duration'],
            video_data['status'],
            video_data['symlink']
        ))

        conn.commit()
        logging.info("Video data inserted successfully.")
    except Exception as e:
        logging.error(f"Error inserting video data into the database: {str(e)}")
    finally:
        conn.close()